In [ ]:
!pip install tensorflow scikit-learn pandas numpy

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

# Sample sequential dataset
# Assuming a time series of monthly data for 4 months
data = {
    'Month': ['2024-01', '2024-02', '2024-03', '2024-04'],
    'Current Loan Amount': [5000, 12000, 15000, 10000],
    'Credit Score': [650, 670, 700, 620],
    'Annual Income': [40000, 42000, 43000, 41000],
    'Monthly Debt': [1000, 1100, 1200, 1050],
    'Risk': [0, 1, 0, 1]  # 0 = No risk, 1 = Risk of fraud/repayment failure
}

# Convert data into a DataFrame
df = pd.DataFrame(data)

# Preprocessing
scaler = StandardScaler()

# Normalize the features
scaled_features = scaler.fit_transform(df[['Current Loan Amount', 'Credit Score', 'Annual Income', 'Monthly Debt']])

# Create time series data for LSTM
X = []
y = []
sequence_length = 2  # Use last 2 months to predict the next month's risk

for i in range(len(scaled_features) - sequence_length):
    X.append(scaled_features[i:i+sequence_length])
    y.append(df['Risk'].iloc[i+sequence_length])

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))  # Sigmoid activation for binary classification (risk or no risk)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"LSTM Model Accuracy: {accuracy * 100:.2f}%")
